In [1]:
set.seed(1919)
library(dplyr)
library(ggplot2)
library(caret)
library(mlbench)
library(ROCR)
library(nnet)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: lattice
Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



In [5]:
dat <- read.table("RFE_data.txt", header = T, sep = "\t")
dat <- as.data.frame(scale(dat))
sample_dat <- read.table("scirep_classes.txt", header = T, sep = ",", stringsAsFactors = F)
sample_class <- as.factor(arrange(sample_dat, sample_id)$label)
merge_dat <- cbind(dat, sample_class)

In [8]:
## Divide Train & Test in Logistic Regression
n_samples <- nrow(merge_dat)
n_train <- floor(n_samples * 0.8)
indices <- sample(1:n_samples)
indices <- indices[1:n_train]
train_merge_sample <- merge_dat[indices,]
#train_sample_class <- sample_class[indices]
test_merge_sample <- merge_dat[-indices,]
#test_sample_class <- sample_class[-indices]

In [9]:
mult_train <- multinom(sample_class ~ ENST00000623130.1 + ENST00000383869.1 + piR.hsa.23317 + ENST00000516053.2 +
            ENST00000626826.1 + ENST00000362154.1 + ENST00000536684.2 + ENST00000384886.3 + ENST00000384278.1+
            ENST00000385273.1, data=train_merge_sample)
step_train <- step(mult_train) 

# weights:  48 (33 variable)
initial  value 210.716743 
iter  10 value 93.687958
iter  20 value 88.718272
iter  30 value 86.627835
iter  40 value 86.359972
final  value 86.359789 
converged
Start:  AIC=238.72
sample_class ~ ENST00000623130.1 + ENST00000383869.1 + piR.hsa.23317 + 
    ENST00000516053.2 + ENST00000626826.1 + ENST00000362154.1 + 
    ENST00000536684.2 + ENST00000384886.3 + ENST00000384278.1 + 
    ENST00000385273.1

trying - ENST00000623130.1 
# weights:  44 (30 variable)
initial  value 210.716743 
iter  10 value 95.320540
iter  20 value 90.894628
iter  30 value 90.535868
iter  40 value 90.526259
iter  40 value 90.526259
iter  40 value 90.526259
final  value 90.526259 
converged
trying - ENST00000383869.1 
# weights:  44 (30 variable)
initial  value 210.716743 
iter  10 value 96.254099
iter  20 value 91.036792
iter  30 value 89.297112
iter  40 value 88.982923
final  value 88.982712 
converged
trying - piR.hsa.23317 
# weights:  44 (30 variable)
initial  value 210.716743 


initial  value 210.716743 
iter  10 value 95.738901
iter  20 value 91.555311
iter  30 value 91.391439
final  value 91.391421 
converged

Step:  AIC=230.78
sample_class ~ ENST00000623130.1 + ENST00000383869.1 + piR.hsa.23317 + 
    ENST00000536684.2 + ENST00000384886.3 + ENST00000384278.1 + 
    ENST00000385273.1

trying - ENST00000623130.1 
# weights:  32 (21 variable)
initial  value 210.716743 
iter  10 value 103.141425
iter  20 value 101.416763
final  value 101.409058 
converged
trying - ENST00000383869.1 
# weights:  32 (21 variable)
initial  value 210.716743 
iter  10 value 98.372905
iter  20 value 94.463965
final  value 94.402078 
converged
trying - piR.hsa.23317 
# weights:  32 (21 variable)
initial  value 210.716743 
iter  10 value 104.499890
iter  20 value 100.211754
iter  30 value 100.167983
iter  30 value 100.167983
iter  30 value 100.167983
final  value 100.167983 
converged
trying - ENST00000536684.2 
# weights:  32 (21 variable)
initial  value 210.716743 
iter  10 value 10

In [11]:
# Test on train_data
train_pred <- predict(step_train)
train_prob_pred <- predict(step_train, type = "prob")
table(train_merge_sample$sample_class, train_pred)

# Test on test_data
test_pred <- predict(mult_train, newdata = test_merge_sample)
test_prob_pred <- predict(mult_train, newdata = test_merge_sample, type = "prob")
table(test_merge_sample$sample_class, test_pred)

                   train_pred
                    Colorectal Cancer Healthy Control Pancreatic Cancer
  Colorectal Cancer                60               9                 0
  Healthy Control                  18              24                 0
  Pancreatic Cancer                 4               0                 0
  Prostate Cancer                   4               0                 0
                   train_pred
                    Prostate Cancer
  Colorectal Cancer               5
  Healthy Control                 0
  Pancreatic Cancer               1
  Prostate Cancer                27

                   test_pred
                    Colorectal Cancer Healthy Control Pancreatic Cancer
  Colorectal Cancer                18               6                 0
  Healthy Control                   3               4                 0
  Pancreatic Cancer                 1               0                 0
  Prostate Cancer                   0               0                 0
                   test_pred
                    Prostate Cancer
  Colorectal Cancer               1
  Healthy Control                 1
  Pancreatic Cancer               0
  Prostate Cancer                 5

In [43]:
## Colorectal Cancer的测试集ROC
positive_class <- 'Colorectal Cancer'
test_labels <- vector('integer', nrow(test_prob_pred))
test_labels[test_merge_sample$sample_class != positive_class] <- 0
test_labels[test_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(test_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Col_Can_ROC_LL_test.png")
plot(roc, main = 'ROC Curve for Colorectal Cancer', sub = 'AUC = 0.803')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.8028571 


In [44]:
## Healthy Control的测试集ROC
positive_class <- 'Healthy Control'
test_labels <- vector('integer', nrow(test_prob_pred))
test_labels[test_merge_sample$sample_class != positive_class] <- 0
test_labels[test_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(test_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Healthy_Control_ROC_LL_test.png")
plot(roc, main = 'ROC Curve for Healthy Control', sub = 'AUC = 0.726')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.7258065 


In [45]:
## Prostate Cancer的测试集ROC
positive_class <- 'Prostate Cancer'
test_labels <- vector('integer', nrow(test_prob_pred))
test_labels[test_merge_sample$sample_class != positive_class] <- 0
test_labels[test_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(test_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Pro_Can_ROC_LL_test.png")
plot(roc, main = 'ROC Curve for Prostate Cancer', sub = 'AUC = 0.976')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.9764706 


In [46]:
## Pancreatic Cancer样本量过小，故在测试集中不再另行绘制其ROC曲线
positive_class <- 'Pancreatic Cancer'
test_labels <- vector('integer', nrow(test_prob_pred))
test_labels[test_merge_sample$sample_class != positive_class] <- 0
test_labels[test_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(test_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
#png("Pan_Can_ROC_LL_test.png")
#plot(roc, main = 'ROC Curve for Pancreatic Cancer', sub = 'AUC = 0.976')
#dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

auc = 0.1052632 


In [47]:
## Healthy Control的训练集ROC
positive_class <- 'Healthy Control'
test_labels <- vector('integer', nrow(train_prob_pred))
test_labels[train_merge_sample$sample_class != positive_class] <- 0
test_labels[train_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(train_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Healthy_Control_ROC_LL_train.png")
plot(roc, main = 'ROC Curve for Healthy Control in Train Set', sub = 'AUC = 0.875')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.8746753 


In [48]:
## Colorectal Cancer的训练集ROC
positive_class <- 'Colorectal Cancer'
test_labels <- vector('integer', nrow(train_prob_pred))
test_labels[train_merge_sample$sample_class != positive_class] <- 0
test_labels[train_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(train_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Col_Can_ROC_LL_train.png")
plot(roc, main = 'ROC Curve for Colorectal Cancer in Train Set', sub = 'AUC = 0.846')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.8459806 


In [49]:
## Prostate Cancer的训练集ROC
positive_class <- 'Prostate Cancer'
test_labels <- vector('integer', nrow(train_prob_pred))
test_labels[train_merge_sample$sample_class != positive_class] <- 0
test_labels[train_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(train_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Pro_Can_ROC_LL_train.png")
plot(roc, main = 'ROC Curve for Prostate Cancer in Train Set', sub = 'AUC = 0.981')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.9813383 


In [50]:
## Pancreatic Cancer的训练集ROC
positive_class <- 'Pancreatic Cancer'
test_labels <- vector('integer', nrow(train_prob_pred))
test_labels[train_merge_sample$sample_class != positive_class] <- 0
test_labels[train_merge_sample$sample_class == positive_class] <- 1
pred <- prediction(train_prob_pred[, positive_class], test_labels)
roc <- performance(pred, 'tpr', 'fpr') 
png("Pan_Can_ROC_LL_train.png")
plot(roc, main = 'ROC Curve for Pancreatic Cancer in Train Set', sub = 'AUC = 0.818')
dev.off()
auc <- performance(pred, 'auc')
cat('auc =', auc@y.values[[1]], '\n')

png 
  2

auc = 0.8176871 
